In [1]:
import os
import sys
import numpy as np
from os.path import abspath, dirname, join, expanduser
from tqdm import trange
CF_DIR = join(expanduser('~'), 'repos', 'celebrity_faces')
DATA_DIR = join(CF_DIR, 'data')
FACENET_SRC_DIR = join(CF_DIR, 'nn_embeddings', 'facenet_src')
sys.path.append(FACENET_SRC_DIR)
from image_processing import *
from lfw import get_paths, read_pairs, evaluate

/Users/yuki/anaconda3/envs/celebrity_faces/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
PAIRS_PATH = join(DATA_DIR, 'pairs.txt')
LFW_RAW_PATH = join(DATA_DIR,  'lfw_raw')

In [3]:
pairs = read_pairs(PAIRS_PATH)
paths, actual_issame = get_paths(LFW_RAW_PATH, pairs)

In [4]:
print('Loading nets for face detection...')
pnet, rnet, onet = get_face_detect_nets()

print('Loading images...')
original_images = [load_image_from_file(path) for path in paths]

Loading nets for face detection...
Loading images...


In [5]:
print('Loading net for embeddings...')
sess, images_placeholder, embeddings, phase_train_placeholder = get_tf_session()

Loading net for embeddings...
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /Users/yuki/repos/celebrity_faces/data/models/20170512-110547/model-20170512-110547.ckpt-250000


In [6]:
nrof_images = len(original_images)
batch_size = 100
nrof_batches = nrof_images // batch_size

In [7]:
embedding_size = embeddings.get_shape()[1]
emb_array = np.zeros((nrof_images, embedding_size))

In [8]:
for i in trange(nrof_batches):
    start_index = i * batch_size
    end_index = min((i + 1) * batch_size, nrof_images)
    images_batch = original_images[start_index:end_index]
    aligned_images = align_data(images_batch, pnet, rnet, onet)
    feed_dict = {images_placeholder: aligned_images, phase_train_placeholder: False}
    emb_array[start_index:end_index, :] = sess.run(embeddings, feed_dict=feed_dict)

100%|██████████| 120/120 [33:43<00:00, 16.86s/it]


In [9]:
print('Evaluating...')
tpr, fpr, accuracy, val, val_std, far = evaluate(emb_array,
                                                 actual_issame,
                                                 nrof_folds=10,
                                                 distance_metric=1,
                                                 subtract_mean=True)

print('Accuracy: %2.5f+-%2.5f' % (np.mean(accuracy), np.std(accuracy)))
print('Validation rate: %2.5f+-%2.5f @ FAR=%2.5f' % (val, val_std, far))

Evaluating...
Accuracy: 0.96983+-0.00353
Validation rate: 0.91300+-0.01531 @ FAR=0.00100
